In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR10",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 10,
            # 'per_task_examples': 10000,
            'per_task_examples': np.inf,
            'per_task_memory_examples': 20,
            'batch_size_train': 128,
            'batch_size_memory': 128,
            'batch_size_validation': 256,
            'tau': 1,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:3' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.001
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=CIFAR10/seed=1_epoch=10_lr=0.001_alpha=0.001_tau=1


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = params['per_task_examples'],
                      random_class_idx = False)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = params['per_task_examples'],
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

elif params['dataset'] == 'CIFAR100':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

[0 1 2 3 4 5 6 7 8 9]
Files already downloaded and verified
Files already downloaded and verified


AttributeError: 'numpy.ndarray' object has no attribute 'clone'

In [ ]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2

# backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
backbone = cl.backbones.ResNet18Small(num_classes_per_head = 2, num_classes = 10, config=params).to(params['device'])
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [4]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
solver=<function LS_solver at 0x7f6f39984700>
[1] Eval metrics for task 1 >> {'accuracy': 83.45, 'loss': 0.0015731319040060044, 'std': 3.4499999999999975}
solver=<function LS_solver at 0x7f6f39984700>
[2] Eval metrics for task 1 >> {'accuracy': 86.1, 'loss': 0.0012974447607994079, 'std': 3.9000000000000035}
solver=<function LS_solver at 0x7f6f39984700>
[3] Eval metrics for task 1 >> {'accuracy': 87.94999999999999, 'loss': 0.0011486100405454635, 'std': 3.9500000000000037}
solver=<function LS_solver at 0x7f6f39984700>
[4] Eval metrics for task 1 >> {'accuracy': 88.1, 'loss': 0.0011259920820593833, 'std': 2.100000000000002}
solver=<function LS_solver at 0x7f6f39984700>
[5] Eval metrics for task 1 >> {'accuracy': 89.35, 'loss': 0.0009576617926359177, 'std': 2.8500000000000023}
solver=<function LS_solver at 0x7f6f39984700>
[6] Eval metrics for task 1 >> {'accuracy': 90.60000000000001, 'loss': 0.0008490382060408593, 'std': 0.3000000

In [5]:
metric_manager_callback.meters['std'].get_std()

[2.2500000000000018,
 21.048916242885287,
 6.083698619169824,
 27.205330360059957,
 17.377928530178735]

In [6]:
metric_manager_callback.meters['accuracy'].get_data()

array([[89.55,  0.  ,  0.  ,  0.  ,  0.  ],
       [71.25, 64.1 ,  0.  ,  0.  ,  0.  ],
       [74.9 , 70.  , 83.95,  0.  ,  0.  ],
       [70.75, 69.9 , 69.75, 50.  ,  0.  ],
       [79.35, 68.55, 65.15, 71.05, 66.1 ]])

In [7]:
metric_manager_callback.meters['accuracy'].compute_overall()

[89.55, 67.675, 76.28333333333333, 65.1, 70.04]

In [8]:
metric_manager_callback.meters['accuracy'].compute_final()

70.04

In [9]:
np.mean(metric_manager_callback.meters['std'].get_std())

14.79317475045876

In [10]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

73.72966666666667